# Limpieza de Datos de Smartwatch: Diagnóstico y Solución

en este cuaderno, seguiremos un proceso estructurado para limpiar un dataset de salud de smartwatch. iniciaremos con un diagnostico completo para entender todos los problemas y, basandonos en los hallazgos, aplicaremos un codigo de limpieza definitivo. la estrategia sera eliminar cualquier registro que presente datos incompletos o problematicos.

## 1. Carga de Librerías y Datos

primero, importaremos las librerias necesarias y cargaremos el conjunto de datos y adicionalmente, ejecutamos un conjunto de comandos para obtener una "radiografia" completa del estado de los datos, esto nos permitira identificar todos los puntos que requieren intervencion.

In [2]:
import pandas as pd
import numpy as np

# cargar el conjunto de datos
df = pd.read_csv("C:/Users/Sam/Downloads/unclean_smartwatch_health_data.csv")


print("--- 1. Informacion General y Tipos de Datos ---")
print(df.info())
print("\n" + "="*50 + "\n")

print("--- 2. Conteo de Valores Nulos por Columna ---")
print(df.isnull().sum())
print("\n" + "="*50 + "\n")

print("--- 3. Muestra de Datos para Inspeccion Visual ---")
print("primeras 10 filas del dataset:")
print(df.head(10))
print("\n" + "="*50 + "\n")

print("--- 4. Estadisticas Descriptivas (para detectar outliers y rangos) ---")
# usamos .describe() para las columnas numericas
print(df.describe())
print("\n" + "="*50 + "\n")

print("--- 5. Analisis de Columnas Categoricas (para detectar inconsistencias) ---")
# usamos .value_counts() para ver los valores unicos en las columnas de texto
print("valores unicos en 'activity level':")
print(df['Activity Level'].value_counts(dropna=False)) # dropna=False para que tambien cuente los nulos
print("\n" + "-"*30 + "\n")
print("valores unicos en 'stress level':")
print(df['Stress Level'].value_counts(dropna=False))

--- 1. Informacion General y Tipos de Datos ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 9799 non-null   float64
 1   Heart Rate (BPM)        9600 non-null   float64
 2   Blood Oxygen Level (%)  9700 non-null   float64
 3   Step Count              9900 non-null   float64
 4   Sleep Duration (hours)  9850 non-null   object 
 5   Activity Level          9800 non-null   object 
 6   Stress Level            9800 non-null   object 
dtypes: float64(4), object(3)
memory usage: 547.0+ KB
None


--- 2. Conteo de Valores Nulos por Columna ---
User ID                   201
Heart Rate (BPM)          400
Blood Oxygen Level (%)    300
Step Count                100
Sleep Duration (hours)    150
Activity Level            200
Stress Level              200
dtype: int64


--- 3. Muestra de Datos para Inspec

## 2. Conclusiones del Diagnóstico

el analisis anterior revela multiples problemas que deben ser solucionados antes de poder usar los datos de manera fiable:

1. **datos nulos generalizados:** todas las columnas presentan una cantidad considerable de valores nulos (nan), lo que indica registros incompletos.
2. **tipos de datos incorrectos:** user id es float64 (decimal) cuando deberia ser un identificador entero. ademas, columnas como sleep duration (hours), activity level y stress level son de tipo object (texto) debido a que contienen valores no numericos.
3. **valores no numericos en columnas numericas:**
    * sleep duration (hours) contiene el texto "error".
    * stress level contiene el texto "very high".
4. **inconsistencias en datos categoricos:** la columna activity level muestra el mismo concepto escrito de varias formas (seddentary vs sedentary, highly active vs highly_active) y con errores de tipeo (actve).
5. **valores atipicos (outliers):** el analisis descriptivo de heart rate (bpm) muestra un valor maximo de 296.59, el cual es fisiologicamente improbable y apunta a un error de medicion.

## 3. Código de Limpieza Definitivo

basado en las conclusiones, procederemos con la limpieza. la estrategia sera estandarizar los datos y luego eliminar por completo cualquier fila que contenga errores, outliers o valores nulos.

In [3]:
# paso 1: estandarizar y marcar errores como nan
# corregir inconsistencias en 'activity level'
df['Activity Level'] = df['Activity Level'].str.lower().str.replace('_', ' ')
df['Activity Level'] = df['Activity Level'].replace({'actve': 'active', 'seddentary': 'sedentary'})

# convertir columnas a numerico, forzando que los errores se marquen como nan
df['Sleep Duration (hours)'] = pd.to_numeric(df['Sleep Duration (hours)'], errors='coerce')

# reemplazar texto y luego convertir a numerico
df['Stress Level'] = df['Stress Level'].replace('Very High', '10')
df['Stress Level'] = pd.to_numeric(df['Stress Level'], errors='coerce')

# marcar outliers de ritmo cardiaco como nan
df.loc[df['Heart Rate (BPM)'] > 220, 'Heart Rate (BPM)'] = np.nan

# paso 2: eliminar todas las filas con cualquier valor nulo
df = df.dropna()

# paso 3: ajustar tipos de datos y eliminar duplicados
df['User ID'] = df['User ID'].astype(int)
df['Step Count'] = df['Step Count'].astype(int)
df['Stress Level'] = df['Stress Level'].astype(int)
df['Heart Rate (BPM)'] = df['Heart Rate (BPM)'].astype(int)

# eliminar filas duplicadas
df = df.drop_duplicates()

## 4. Verificación Final

finalmente, verificamos que el dataframe este completamente limpio, sin valores nulos y con los tipos de datos correctos.

In [4]:
print(df.info())
print("\nConteo de nulos restantes:")
print(df.isnull().sum())
print("\nMuestra final de los datos:")
print(df.head())

<class 'pandas.core.frame.DataFrame'>
Index: 8295 entries, 0 to 9999
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   User ID                 8295 non-null   int64  
 1   Heart Rate (BPM)        8295 non-null   int64  
 2   Blood Oxygen Level (%)  8295 non-null   float64
 3   Step Count              8295 non-null   int64  
 4   Sleep Duration (hours)  8295 non-null   float64
 5   Activity Level          8295 non-null   object 
 6   Stress Level            8295 non-null   int64  
dtypes: float64(2), int64(4), object(1)
memory usage: 518.4+ KB
None

Conteo de nulos restantes:
User ID                   0
Heart Rate (BPM)          0
Blood Oxygen Level (%)    0
Step Count                0
Sleep Duration (hours)    0
Activity Level            0
Stress Level              0
dtype: int64

Muestra final de los datos:
   User ID  Heart Rate (BPM)  Blood Oxygen Level (%)  Step Count  \
0     4174         

## 6. Conclusión de la Limpieza

tras ejecutar el codigo de limpieza y verificar el estado final del dataframe, podemos confirmar que el proceso ha sido exitoso. el resultado es un dataset robusto, consistente y listo para un analisis fiable.

el proceso partio de un diagnostico que revelo 5 problematicas criticas: datos nulos, tipos de datos incorrectos, valores no numericos, inconsistencias categoricas y outliers. en lugar de imputar o adivinar los datos faltantes, se opto por una estrategia de eliminacion para garantizar la maxima integridad de los datos restantes.

como resultado, el dataset se redujo de 10,000 a 8,295 registros, eliminando todas las filas que presentaban informacion incompleta o erronea.